In [ ]:
import numpy as np

In [ ]:
class Map(object):
    def __init__(self) -> None:
        self._map_arr = np.array(Image.open(map_path).fli)
        self._y, self._x, _ = self._map_arr.shape
        self._win_size = 1024
        self._win_center = (512, self._y - 1 - (self._win_size / 2))
    
    def _get_fov(self, loc):
        fov = self._map_arr[0:self._win_size, self._x - int(self._win_size): self._y] 

        print(self._map_arr.shape, fov.shape)
        return fov

    def _preprocess(self, arr):
        return pygame.surfarray.make_surface(arr)

    def get_map(self, loc) -> pygame.Surface:
        return self._preprocess(self._get_fov(loc))

    @property
    def map(self) -> np.array:
        return self._map_arr
